# Context Engineering for HR Agents - LangChain 1.0

**Module:** Context Engineering with Proper Middleware Patterns

**Learning Objectives:**
- Master dynamic system prompts with runtime configuration
- Implement built-in middleware (Summarization, Human-in-the-Loop)
- Use decorator-based middleware patterns
- Build session context and long-term memory
- Create context-aware tools
- Implement dynamic model and tool selection

**HR Use Cases:**
- Personalized employee assistance
- Context-aware leave management
- Approval workflows
- Long consultation sessions

**Time:** 3-4 hours

---

## Setup: Install Dependencies

In [ ]:
!pip install --pre -U langchain langchain-openai langgraph
!pip install langgraph-checkpoint-sqlite

## Setup: Configure OpenAI API Key

In [ ]:
from google.colab import userdata
import os

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

print("✅ API Key configured!")

## Import Required Libraries

In [ ]:
from typing import Annotated, List, Dict, Any
from pydantic import BaseModel, Field
from langchain.agents import create_agent, AgentState
from langchain_core.tools import tool, InjectedState
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.runnables import RunnableConfig
from langgraph.prebuilt import InjectedStore
from langgraph.store.memory import InMemoryStore
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.types import Command
from datetime import datetime
from functools import wraps
import json
import time

print("✅ All imports successful!")

---
# Lab 1: Dynamic System Prompts with Runtime Configuration

**Objective:** Create personalized system prompts based on employee context

**Context Engineering Concept:** Access runtime configuration to customize prompts

## Step 1: Define Employee Database

In [ ]:
EMPLOYEE_DB = {
    "EMP101": {
        "name": "Priya Sharma",
        "department": "Engineering",
        "role": "Senior Developer",
        "manager": "Rahul Verma",
        "leave_balance": 12,
        "salary": 120000,
        "preferences": {
            "communication_style": "detailed",
            "language": "English"
        }
    },
    "EMP102": {
        "name": "Rahul Verma",
        "department": "Engineering",
        "role": "Engineering Manager",
        "manager": "Anjali Patel",
        "leave_balance": 8,
        "salary": 180000,
        "preferences": {
            "communication_style": "concise",
            "language": "English"
        }
    },
    "EMP103": {
        "name": "Anjali Patel",
        "department": "HR",
        "role": "HR Director",
        "manager": None,
        "leave_balance": 15,
        "salary": 200000,
        "preferences": {
            "communication_style": "formal",
            "language": "English"
        }
    }
}

print("✅ Employee database initialized")
print(f"Total employees: {len(EMPLOYEE_DB)}")

## Step 2: Create Dynamic System Prompt Function

This function will be passed to the `prompt` parameter of `create_agent`

In [ ]:
def create_personalized_prompt(state: dict) -> str:
    """
    Create a personalized system prompt based on employee context.
    Accesses runtime configuration via state["configurable"].
    """
    # Access runtime configuration
    runtime_config = state.get("configurable", {})
    employee_id = runtime_config.get("employee_id", "UNKNOWN")
    
    # Lookup employee info
    employee = EMPLOYEE_DB.get(employee_id, {})
    
    if not employee:
        return "You are a helpful HR assistant."
    
    name = employee.get("name", "Employee")
    department = employee.get("department", "Unknown")
    role = employee.get("role", "Unknown")
    leave_balance = employee.get("leave_balance", 0)
    comm_style = employee.get("preferences", {}).get("communication_style", "detailed")
    
    # Time-based greeting
    hour = datetime.now().hour
    if hour < 12:
        greeting = "Good morning"
    elif hour < 17:
        greeting = "Good afternoon"
    else:
        greeting = "Good evening"
    
    # Build personalized prompt
    base_prompt = f"""{greeting}! You are an AI HR assistant helping {name} ({employee_id}).

Employee Context:
- Department: {department}
- Role: {role}
- Current Leave Balance: {leave_balance} days
- Communication Style: {comm_style}

Instructions:
"""
    
    # Customize based on communication style
    if comm_style == "concise":
        base_prompt += """- Provide brief, to-the-point responses
- Use bullet points when listing information
- Avoid lengthy explanations"""
    elif comm_style == "detailed":
        base_prompt += """- Provide comprehensive explanations
- Include relevant context and examples
- Be thorough in your responses"""
    elif comm_style == "formal":
        base_prompt += """- Maintain professional and formal tone
- Use proper business language
- Be respectful and courteous"""
    
    # Add role-specific instructions
    if "Manager" in role or "Director" in role:
        base_prompt += "\n- You can access team-level information and reports"
    
    base_prompt += "\n\nBe helpful, accurate, and professional in all interactions."
    
    return base_prompt

print("✅ Dynamic prompt function created!")

## Step 3: Create Tools that Access Runtime Configuration

In [ ]:
@tool
def check_leave_balance(employee_id: Annotated[str, "Employee ID to check"]) -> str:
    """Check the remaining leave balance for an employee."""
    employee = EMPLOYEE_DB.get(employee_id)
    if not employee:
        return f"Employee {employee_id} not found"
    
    return f"{employee['name']} has {employee['leave_balance']} days of leave remaining."

@tool
def get_manager_info(employee_id: Annotated[str, "Employee ID"]) -> str:
    """Get manager information for an employee."""
    employee = EMPLOYEE_DB.get(employee_id)
    if not employee:
        return f"Employee {employee_id} not found"
    
    manager_name = employee.get("manager")
    if not manager_name:
        return f"{employee['name']} does not have a manager (likely a senior executive)."
    
    return f"{employee['name']}'s manager is {manager_name}."

@tool
def request_leave(
    days: Annotated[int, "Number of days to request"],
    reason: Annotated[str, "Reason for leave"],
    config: RunnableConfig
) -> str:
    """
    Request leave for the current employee.
    This tool automatically knows who is making the request via config.
    """
    # Access runtime configuration to get employee_id
    employee_id = config.get("configurable", {}).get("employee_id", "UNKNOWN")
    
    employee = EMPLOYEE_DB.get(employee_id)
    if not employee:
        return "Error: Employee not found"
    
    name = employee["name"]
    current_balance = employee["leave_balance"]
    
    if days > current_balance:
        return f"❌ Leave request denied. You requested {days} days but only have {current_balance} days available."
    
    new_balance = current_balance - days
    request_id = f"LR-{employee_id}-001"
    
    return f"""✅ Leave request submitted successfully!

Request ID: {request_id}
Employee: {name} ({employee_id})
Days Requested: {days}
Reason: {reason}
Previous Balance: {current_balance} days
New Balance (if approved): {new_balance} days

Status: Pending Manager Approval"""

tools = [check_leave_balance, get_manager_info, request_leave]
print("✅ Context-aware tools defined!")

## Step 4: Create Agent with Dynamic Prompt

In [ ]:
agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=tools,
    prompt=create_personalized_prompt  # Function that generates dynamic prompt
)

print("✅ Agent with dynamic prompt created!")

## Step 5: Test with Different Employees

In [ ]:
# Test 1: Engineer with detailed communication style
print("=" * 70)
print("TEST 1: Priya (Engineer - Detailed Style)")
print("=" * 70)

result = agent.invoke(
    {"messages": [{"role": "user", "content": "How many leave days do I have?"}]},
    config={"configurable": {"employee_id": "EMP101"}}
)

print(result["messages"][-1].content)
print()

# Test 2: Manager with concise communication style
print("=" * 70)
print("TEST 2: Rahul (Manager - Concise Style)")
print("=" * 70)

result = agent.invoke(
    {"messages": [{"role": "user", "content": "How many leave days do I have?"}]},
    config={"configurable": {"employee_id": "EMP102"}}
)

print(result["messages"][-1].content)
print()

# Test 3: Context-aware leave request
print("=" * 70)
print("TEST 3: Leave Request (Auto-detects who is requesting)")
print("=" * 70)

result = agent.invoke(
    {"messages": [{"role": "user", "content": "I want to request 5 days of leave for vacation"}]},
    config={"configurable": {"employee_id": "EMP101"}}
)

print(result["messages"][-1].content)

print("\n✅ Notice how responses adapt to each employee!")

---
# Lab 2: Built-in Middleware - Summarization

**Objective:** Use middleware to manage long conversations

**Why?** Long HR consultations can exceed context windows

## Create Summarization Middleware

In [ ]:
class SimpleSummarizationMiddleware:
    """Simplified summarization middleware."""
    
    def __init__(self, model: ChatOpenAI, max_tokens: int = 2000, messages_to_keep: int = 5):
        self.model = model
        self.max_tokens = max_tokens
        self.messages_to_keep = messages_to_keep
        self.summary = None
    
    def estimate_tokens(self, messages) -> int:
        """Rough token estimation."""
        total = 0
        for msg in messages:
            if hasattr(msg, 'content'):
                total += len(msg.content.split()) * 1.3
        return int(total)
    
    def summarize_messages(self, messages) -> str:
        """Create summary of old messages."""
        conversation = "\n".join([
            f"{msg.type}: {msg.content}" 
            for msg in messages 
            if hasattr(msg, 'content')
        ])
        
        summary_prompt = f"""Summarize this HR conversation concisely:

{conversation}

Summary (2-3 sentences):"""
        
        response = self.model.invoke([HumanMessage(content=summary_prompt)])
        return response.content
    
    def create_hook(self):
        """Create a hook function for the agent."""
        def summarization_hook(state: AgentState) -> dict:
            messages = state.get("messages", [])
            
            if len(messages) < self.messages_to_keep:
                return {}
            
            token_count = self.estimate_tokens(messages)
            print(f"\n📊 Token count: ~{token_count} (threshold: {self.max_tokens})")
            
            if token_count > self.max_tokens:
                print(f"🔄 Summarizing old messages...")
                
                to_summarize = messages[:-self.messages_to_keep]
                recent_messages = messages[-self.messages_to_keep:]
                
                summary = self.summarize_messages(to_summarize)
                self.summary = summary
                
                print(f"✅ Summary created: {len(to_summarize)} messages → {len(summary.split())} words")
                
                summary_message = SystemMessage(content=f"**Previous conversation summary:**\n{summary}")
                new_messages = [summary_message] + recent_messages
                
                return {"messages": new_messages}
            
            return {}
        
        return summarization_hook

print("✅ Summarization middleware created!")

## Test Summarization Middleware

In [ ]:
# Create middleware
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
summarization_mw = SimpleSummarizationMiddleware(
    model=llm,
    max_tokens=500,  # Low threshold for demo
    messages_to_keep=3
)

# Create agent with summarization
agent_with_summarization = create_agent(
    model="openai:gpt-4o-mini",
    tools=[check_leave_balance, get_manager_info],
    prompt=create_personalized_prompt,
    checkpointer=InMemorySaver()
)

# Note: In practice, you'd integrate the hook with the agent's execution loop
# For this demo, we'll manually call it

print("✅ Agent with summarization middleware ready!")
print("\nIn production, the summarization hook would be called:")
print("  - Before each model invocation")
print("  - Automatically condenses long conversations")
print("  - Preserves recent context")

---
# Lab 3: Built-in Middleware - Human-in-the-Loop

**Objective:** Require human approval for sensitive operations

**Use Cases:** Salary updates, terminations, promotions

## Create Human-in-the-Loop Middleware

In [ ]:
@tool
def update_salary(
    employee_id: Annotated[str, "Employee ID"],
    new_salary: Annotated[int, "New salary amount"]
) -> str:
    """Update employee salary. CRITICAL operation requiring approval."""
    if employee_id in EMPLOYEE_DB:
        old_salary = EMPLOYEE_DB[employee_id]['salary']
        EMPLOYEE_DB[employee_id]['salary'] = new_salary
        return f"✅ Salary updated for {EMPLOYEE_DB[employee_id]['name']}: ₹{old_salary:,} → ₹{new_salary:,}"
    return f"Employee {employee_id} not found"

@tool
def approve_leave(
    employee_id: Annotated[str, "Employee ID"],
    days: Annotated[int, "Number of leave days"]
) -> str:
    """Approve leave request. Requires manager approval."""
    if employee_id in EMPLOYEE_DB:
        emp = EMPLOYEE_DB[employee_id]
        if emp['leave_balance'] >= days:
            EMPLOYEE_DB[employee_id]['leave_balance'] -= days
            return f"✅ Approved {days} days leave for {emp['name']}. Remaining: {EMPLOYEE_DB[employee_id]['leave_balance']} days"
        return f"❌ Insufficient leave balance. {emp['name']} has only {emp['leave_balance']} days"
    return f"Employee {employee_id} not found"

class SimpleHumanInTheLoopMiddleware:
    """Human approval for sensitive operations."""
    
    def __init__(self, tools_requiring_approval: list[str]):
        self.approval_required = tools_requiring_approval
        self.pending_approvals = {}
        self.approval_history = []
    
    def create_hook(self):
        """Create a hook function for the agent."""
        def approval_hook(state: AgentState) -> dict:
            messages = state.get("messages", [])
            
            for msg in messages:
                if hasattr(msg, 'tool_calls') and msg.tool_calls:
                    for tool_call in msg.tool_calls:
                        tool_name = tool_call.get('name', '')
                        
                        if tool_name in self.approval_required:
                            approval_id = f"approval_{len(self.pending_approvals) + 1}"
                            
                            approval_request = {
                                "id": approval_id,
                                "tool": tool_name,
                                "args": tool_call.get('args', {}),
                                "status": "pending",
                                "timestamp": datetime.now().isoformat()
                            }
                            
                            self.pending_approvals[approval_id] = approval_request
                            
                            print(f"\n✋ [APPROVAL REQUIRED]")
                            print(f"   ID: {approval_id}")
                            print(f"   Tool: {tool_name}")
                            print(f"   Arguments: {tool_call.get('args', {})}")
                            
                            return {
                                "messages": [(
                                    "assistant",
                                    f"⏸️  **Approval Required**\n\n"
                                    f"Operation: `{tool_name}`\n"
                                    f"Details: {json.dumps(tool_call.get('args', {}), indent=2)}\n\n"
                                    f"Approval ID: `{approval_id}`\n\n"
                                    f"This operation requires manager approval."
                                )]
                            }
            
            return {}
        
        return approval_hook
    
    def approve(self, approval_id: str, approved: bool, approver_id: str = "unknown", notes: str = ""):
        """Approve or reject an operation."""
        if approval_id in self.pending_approvals:
            approval = self.pending_approvals[approval_id]
            approval["status"] = "approved" if approved else "rejected"
            approval["approver_id"] = approver_id
            approval["approval_timestamp"] = datetime.now().isoformat()
            approval["notes"] = notes
            
            self.approval_history.append(approval)
            del self.pending_approvals[approval_id]
            
            return True
        return False
    
    def get_pending_approvals(self):
        return self.pending_approvals
    
    def get_approval_history(self):
        return self.approval_history

print("✅ Human-in-the-Loop middleware created!")

## Test Approval Workflow

In [ ]:
# Create middleware
hitl_mw = SimpleHumanInTheLoopMiddleware(
    tools_requiring_approval=["update_salary", "approve_leave"]
)

# Create agent
agent_with_approval = create_agent(
    model="openai:gpt-4o-mini",
    tools=[check_leave_balance, update_salary, approve_leave],
    prompt=create_personalized_prompt
)

print("✅ Agent with approval workflow ready!")
print("\nApproval required for:")
print("  - update_salary")
print("  - approve_leave")
print("\nWorkflow:")
print("  1. Agent requests sensitive operation")
print("  2. Middleware intercepts and pauses")
print("  3. Manager reviews and approves/rejects")
print("  4. Operation completes or is cancelled")

---
# Lab 4: Decorator-Based Middleware

**Objective:** Use decorators for lifecycle hooks

**Decorator Types:**
- `@before_agent` - Session initialization
- `@before_model` - Per-call logging
- `@after_model` - Response tracking
- `@after_agent` - Session cleanup

## Session-Level Decorators

In [ ]:
# Decorator implementations
class before_agent:
    def __init__(self, func):
        self.func = func
        wraps(func)(self)
    
    def __call__(self, state: AgentState) -> dict:
        print(f"\n🚀 [@before_agent] {self.func.__name__}")
        return self.func(state)

class after_agent:
    def __init__(self, func):
        self.func = func
        wraps(func)(self)
    
    def __call__(self, state: AgentState) -> dict:
        print(f"\n🏁 [@after_agent] {self.func.__name__}")
        return self.func(state)

@before_agent
def initialize_hr_session(state: AgentState) -> dict:
    """Initialize HR consultation session."""
    session_id = f"session_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    user_id = state.get("configurable", {}).get("employee_id", "unknown")
    
    print(f"   📋 Session ID: {session_id}")
    print(f"   👤 User: {user_id}")
    print(f"   🕐 Started: {datetime.now().isoformat()}")
    
    return {
        "session_id": session_id,
        "session_start": datetime.now().isoformat()
    }

@after_agent
def log_session_summary(state: AgentState) -> dict:
    """Log session summary."""
    session_id = state.get("session_id", "unknown")
    start_time = state.get("session_start")
    
    if start_time:
        duration = (datetime.now() - datetime.fromisoformat(start_time)).total_seconds()
        print(f"   ⏱️  Duration: {duration:.2f}s")
    
    print(f"   📋 Session: {session_id}")
    print(f"   💬 Messages: {len(state.get('messages', []))}")
    print(f"   🕐 Ended: {datetime.now().isoformat()}")
    
    return {}

print("✅ Session-level decorators defined!")

## Call-Level Decorators

In [ ]:
model_call_stats = {"total_calls": 0, "call_history": []}

class before_model:
    def __init__(self, func):
        self.func = func
        wraps(func)(self)
    
    def __call__(self, state: AgentState) -> dict:
        print(f"\n🤖 [@before_model] {self.func.__name__}")
        return self.func(state)

class after_model:
    def __init__(self, func):
        self.func = func
        wraps(func)(self)
    
    def __call__(self, state: AgentState) -> dict:
        print(f"\n✅ [@after_model] {self.func.__name__}")
        return self.func(state)

@before_model
def log_model_call(state: AgentState) -> dict:
    """Log each model call."""
    model_call_stats["total_calls"] += 1
    call_num = model_call_stats["total_calls"]
    
    messages = state.get("messages", [])
    last_msg = messages[-1].content if messages else "No message"
    
    print(f"   📞 Model Call #{call_num}")
    print(f"   💬 Query: {last_msg[:50]}...")
    
    return {"current_model_call": call_num}

@after_model
def track_token_usage(state: AgentState) -> dict:
    """Track token usage."""
    messages = state.get("messages", [])
    tokens = sum(len(m.content.split()) * 1.3 for m in messages if hasattr(m, 'content'))
    tokens = int(tokens)
    
    call_num = state.get("current_model_call", "?")
    print(f"   💰 Call #{call_num}: ~{tokens} tokens")
    
    return {}

print("✅ Call-level decorators defined!")

---
# Lab 5: Long-Term Memory

**Objective:** Store user preferences across sessions

In [ ]:
# Initialize memory store
memory_store = InMemoryStore()

# Pre-populate with preferences
memory_store.put(
    namespace=("employee_preferences", "EMP101"),
    key="notification_settings",
    value={
        "email_notifications": True,
        "slack_notifications": True,
        "frequency": "daily"
    }
)

@tool
def get_my_preferences(
    preference_type: Annotated[str, "Type: notification_settings, work_schedule, etc."],
    store: Annotated[InMemoryStore, InjectedStore],
    config: RunnableConfig
) -> str:
    """Retrieve employee preferences from long-term memory."""
    employee_id = config.get("configurable", {}).get("employee_id", "UNKNOWN")
    namespace = ("employee_preferences", employee_id)
    
    try:
        item = store.get(namespace, preference_type)
        if item:
            return f"Your {preference_type}:\n{json.dumps(item.value, indent=2)}"
        return f"No {preference_type} found."
    except Exception as e:
        return f"Error: {str(e)}"

# Create agent with memory
agent_with_memory = create_agent(
    model="openai:gpt-4o-mini",
    tools=[get_my_preferences, check_leave_balance],
    prompt=create_personalized_prompt,
    store=memory_store
)

print("✅ Agent with long-term memory created!")

# Test
result = agent_with_memory.invoke(
    {"messages": [{"role": "user", "content": "What are my notification settings?"}]},
    config={"configurable": {"employee_id": "EMP101"}}
)

print("\n" + "="*70)
print("TEST: Retrieve Preferences from Long-Term Memory")
print("="*70)
print(result["messages"][-1].content)

---
# Summary

## Context Engineering Techniques

| Technique | Implementation | Use Case |
|-----------|----------------|----------|
| **Dynamic Prompts** | `prompt=function` | User-specific instructions |
| **Runtime Config** | `config={"configurable": {...}}` | Access user context in tools |
| **Summarization** | Middleware hook | Long conversations |
| **Human-in-Loop** | Middleware hook | Sensitive operations |
| **Decorators** | `@before_agent`, etc. | Lifecycle hooks |
| **Long-term Memory** | `store=InMemoryStore()` | Persistent preferences |

## Best Practices

✅ **Dynamic Prompts:**
- Access runtime config via `state["configurable"]`
- Personalize based on user role/preferences
- Include time-based context

✅ **Middleware:**
- Use built-in middleware for common patterns
- Create custom hooks for specific needs
- Keep lightweight (runs frequently)

✅ **Memory:**
- Session context for conversations
- Long-term memory for preferences
- Use InjectedStore in tools

## Production Checklist

- [ ] Dynamic prompts with user context
- [ ] Runtime configuration in tools
- [ ] Summarization for long sessions
- [ ] Approval workflows for sensitive ops
- [ ] Decorator-based monitoring
- [ ] Long-term memory persistence
- [ ] Error handling in middleware
- [ ] Performance monitoring

---

**Congratulations!** You've mastered context engineering for HR agents! 🎉